In [1]:
# Initial imports
import os
import pandas as pd
from dotenv import load_dotenv
import nltk as nltk
import requests
import os
import json
nltk.download('vader_lexicon')
from nltk.sentiment.vader import SentimentIntensityAnalyzer
analyzer = SentimentIntensityAnalyzer()
from path import Path
from newsapi import NewsApiClient
import spacy
from spacy import displacy

%matplotlib inline

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\benja\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


## 1. Qualitative Analysis - taking NewsAPI articles on the Canadian Housing Market 

Use the [newsapi](https://newsapi.org/) to pull the latest news articles for the Canadian Housing Market

In [2]:
# Read your api key environment variable
# Load environment variables and retrieve the News API key
load_dotenv()
api_key = os.getenv("NEWS_API_KEY")

In [3]:
# Create the newsapi client
newsapi = NewsApiClient(api_key=api_key)

In [4]:
nlp = spacy.load('en_core_web_sm')

In [5]:
# Fetch the Bitcoin news articles
housingmarket_news_en = newsapi.get_everything(
    q="Canadian Housing Market",
    language="en"
)

# Show the total number of news
housingmarket_news_en["totalResults"]

157

## 1. Qualitative Analysis - taking Twitter articles on the Canadian Housing Market 

Use the Twitter APIv2 to pull the latest news articles for the Canadian Housing Market

In [8]:
import requests
import os
import json

# To set your environment variables in your terminal run the following line:
# export 'BEARER_TOKEN'='<your_bearer_token>'
bearer_token = os.getenv("twitter_bearer_token")

search_url = "https://api.twitter.com/2/tweets/search/recent"

# Optional params: start_time,end_time,since_id,until_id,max_results,next_token,
# expansions,tweet.fields,media.fields,poll.fields,place.fields,user.fields
query_params = {'query': '("Canadian Housing Market") -"recent search" OR #twitterdev','tweet.fields': 'author_id'}



In [9]:
def bearer_oauth(r):
    """
    Method required by bearer token authentication.
    """

    r.headers["Authorization"] = f"Bearer {bearer_token}"
    r.headers["User-Agent"] = "v2RecentSearchPython"
    return r

def connect_to_endpoint(url, params):
    response = requests.get(url, auth=bearer_oauth, params=params)
    print(response.status_code)
    if response.status_code != 200:
        raise Exception(response.status_code, response.text)
    return response.json()


def main():
    json_response = connect_to_endpoint(search_url, query_params)
    print(json.dumps(json_response, indent=4, sort_keys=True))


if __name__ == "__main__":
    main()

200
{
    "data": [
        {
            "author_id": "21298976",
            "id": "1465703559610048519",
            "text": "WOW - The number of people adding at least a fourth mortgage advanced 7.7 per cent, doubling the figure among first-time borrowers in the 12 months ending in June.\n\nhttps://t.co/ftZNmjqNyo\n\n#mortgages #realestate #realestateinvesting https://t.co/8qqyt78lHl"
        },
        {
            "author_id": "794190151551512576",
            "id": "1465691296056721410",
            "text": "\"Canada is addicted to mortgage debt.\"\n\nWhen #interestRates rise, be sure you have your #mortgage strategy in place. \n\nDon't have a mortgage strategy? Let's plan one for you.\n\nhttps://t.co/ZAueZTL0jl"
        },
        {
            "author_id": "403751813",
            "id": "1465651043715407884",
            "text": "Investors Remain Active in the Hot #Canadian #Housing Market https://t.co/Upa6xVrRF0 #RealEstate #Investing #Mortgage #Realtor #property #REINCanada